In [1]:
# Model Arguments 
from config import ModelArguments

config = ModelArguments()

# Multimodal Encoder, Resampler, Projector
from multimodal_encoder.builder import build_vision_tower
from multimodal_resampler.builder import build_vision_resampler
from multimodal_projector.builder import build_vision_projector

vision_tower = build_vision_tower(config)
vision_resampler = build_vision_resampler(config)
vision_projector = build_vision_projector(config)

On MacOS, decord package is not compatible with new python version.


/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from llava_llama import LlavaLlamaForCausalLM # Register the llava models into 'transformers'
from transformers import AutoConfig
from transformers import AutoModelForCausalLM

config = AutoConfig.for_model("llava_llama", trust_remote_code=True)
model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)

: 

In [2]:
# Dataset Construction for MultiModality

from dataprocess import LazySupervisedDataset, DataCollatorForSupervisedDataset
from torch.utils.data import DataLoader
from config import DataArguments 
from llava_llama import prep_llava_llama_tokenizer

data_args = DataArguments(
    data_path = "data/mock.json",
    image_folder = "data/",
    video_folder = "data/",
    video_fps = 1,
    frames_upbound = 0,
    add_time_instruction = False,
    force_sample = False,
    default_fps = 10
)

tokenizer = prep_llava_llama_tokenizer("meta-llama/Meta-Llama-3.1-8B-Instruct")
dataset = LazySupervisedDataset(data_args=data_args, tokenizer=tokenizer, image_processor=vision_tower.image_processor)
collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

dataloader = DataLoader(dataset, collate_fn=collator, batch_size=2, num_workers=1)


data_dict = dataset[0]

# data_batch = next(iter(dataloader))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
dataset[0]['image'][1][1]

tensor([[0, 1],
        [0, 1],
        [0, 1],
        [0, 1]])

In [4]:
dataset[0]['image']

[(tensor([[[[-1.0623, -1.2521, -1.4127,  ..., -0.3178, -0.3032, -0.3178],
            [-1.0623, -1.2083, -1.3543,  ..., -0.6390, -0.6244, -0.5368],
            [-1.0623, -1.1645, -1.2813,  ..., -0.5514, -0.6390, -0.7412],
            ...,
            [ 1.2150,  1.2150,  1.2442,  ..., -1.0623, -0.9018, -0.7120],
            [ 1.2150,  1.2296,  1.2296,  ..., -1.1353, -1.0039, -0.8580],
            [ 1.2442,  1.2296,  1.2150,  ..., -1.1353, -1.0331, -0.9018]],
  
           [[-0.8666, -1.0617, -1.2268,  ..., -0.3864, -0.3864, -0.4164],
            [-0.8666, -1.0167, -1.1668,  ..., -0.6715, -0.6715, -0.6115],
            [-0.8516, -0.9717, -1.0918,  ..., -0.5815, -0.6715, -0.7766],
            ...,
            [ 1.2194,  1.2194,  1.2495,  ..., -1.0617, -0.9117, -0.7616],
            [ 1.2194,  1.2344,  1.2344,  ..., -1.1368, -1.0167, -0.9117],
            [ 1.2344,  1.2194,  1.2044,  ..., -1.1368, -1.0467, -0.9717]],
  
           [[-1.0821, -1.2385, -1.3665,  ..., -0.3568, -0.3568, -0.385

In [9]:
# data_batch["images"]

In [ ]:
padded_img = sum([img.sum()==0 for img in images[0]])
padded_modality = [mod.sum()==0 for mod in modalities[0]]
assert padded_img == padded_modality, "Modality mismatch with Image in padded number"

In [ ]:
# Right, now we try to propagate the data_dict into the model 
import torch 
from constants import IGNORE_INDEX

# Prepare the inputs
input_ids = data_dict['input_ids'].unsqueeze(0)  # Add batch dimension
labels = data_dict['labels'].unsqueeze(0)  # Add batch dimension

# Prepare images, a list of per-patch-images tensors
images = []
if 'image' in data_dict:
    batch_images = [item[0] for item in data_dict["image"]]
    batch_images = torch.cat(batch_images, dim=0)
    images = [batch_images]

# Create attention mask
attention_mask = torch.ones_like(input_ids, dtype=torch.bool)

# Forward pass
outputs = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    labels=labels,
    images=images,
)